<img style="float: right;" width="120" src="../Images/supplier-logo.png">
<img style="float: left; margin-top: 0" width="80" src="../Images/client-logo.png">
<br><br><br>

# Pivots

## Introduction

Most people likely have experience with pivot tables in Excel. Pandas provides a similar function called (appropriately enough) pivot_table . While it is exceedingly useful, I frequently find myself struggling to remember how to use the syntax to format the output for my needs. This notebook will focus on explaining the pandas `pivot_table` function and how to use it for your data analysis.



## The Data

One of the challenges with using the panda’s pivot_table is making sure you understand your data and what questions you are trying to answer with the pivot table. It is a seemingly simple function but can produce very powerful analysis very quickly.

THis example is going to track a sales pipeline (also called funnel). The basic problem is that some sales cycles are very long (think “enterprise software”, capital equipment, etc.) and users might want to understand it in more detail throughout the year.

Typical questions include:

> 
> How much revenue is in the pipeline? <BR>
> What products are in the pipeline?<BR>
> Who has what products at what stage?<BR>
> How likely are we to close deals by year end?<BR>
>
Many companies will have CRM tools or other software that sales uses to track the process. While they may have useful tools for analyzing the data, inevitably someone will export the data to Excel and use a PivotTable to summarize the data.

Using a panda’s pivot table can be a good alternative because it is:

- Quicker (once it is set up)
- Self documenting (look at the code and you know what it does)
- Easy to use to generate a report or email
- More flexible because you can define custome aggregation functions



## Read in the Data

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_excel(io = "../Data/sample_data.xls", sheet_name="Sales")
display(df.head())
df.dtypes

## Define a Category

For convenience only, define the `status` column as a category and set the order we want to view.

Not strictly required but helps to keep the order we want as we work through analyzing the data.

Note the change in type for the `Status` column. It was an object and is now a category.

In [ ]:
df["Status"] = df["Status"].astype("category")
df["Status"].cat.set_categories(["won","pending","presented","declined"],inplace=True)

display(df.head())
df.dtypes

## Pivot the data - single Index

Generally best to pivot one step at a time. Add items and check each step to verify you are getting the results expected. Don’t be afraid to play with the order and the variables to see what presentation makes the most sense for your needs.

The simplest pivot table **must** have a dataframe and an index . In this case, use the `Name` column as the index.

**Note** the only columns in the pivto table are the numeric ones

In [ ]:
pd.pivot_table(df,index=["Name"])

## Pivot the data - Multiple Indices

You can have multiple indexes as well. In fact, most of the pivot_table args can take multiple values via a list.

In [ ]:
pd.pivot_table(df,index=["Name","Rep","Manager"])

Look at this by Manager and Rep. It’s easy enough to do by changing the index .

In [ ]:
pd.pivot_table(df,index=["Manager","Rep"])

## Aggrating and Summarizing Data

The pivot table is smart enough to start aggregating the data and summarizing it by grouping the reps with their managers. 

For this purpose, the `Account` and `Quantity` columns aren’t really useful. Let’s remove it by explicitly defining the columns we care about using the values field.

In [ ]:
pd.pivot_table(df,index=["Manager","Rep"],values=["Price"])

The price column automatically averages the data (calcultes the mean) but we can do a `count` or a `sum`. Adding them is simple using aggfunc and np.sum .


In [ ]:
pd.pivot_table(df,index=["Manager","Rep"],values=["Price"],aggfunc=np.sum)

The `aggfunc` parameter can take a list of functions. E.g. using the numpy `mean` function and `len` to get a count.

In [ ]:
funcs = [np.mean,len]
pd.pivot_table(df,index=["Manager","Rep"],values=["Price"],aggfunc=funcs)

The `aggfunc` parameter can take a list of names of functions. <BR>
E.g. using the numpy `mean` function and `std` to get a count.

In [ ]:
funcs = ['mean', 'std']
pd.pivot_table(df,index=["Manager","Rep"],values=["Price"],aggfunc=funcs)

If we want to see sales broken down by the products, the `columns` paramneter allows you to define one or more columns.

## Columns vs. Values
One of the confusing points with the pivot_table is the use of columns and values .  
Remember, columns are optional - they provide an additional way to segment the actual values you care about.  
The aggregation functions are applied to the **values** you list.

In [ ]:
pd.pivot_table(df,index=["Manager","Rep"],values=["Price"],
               columns=["Product"],aggfunc=[np.sum])

Remove the Nans bu using the `fill_value` to set them to 0.

In [ ]:
pd.pivot_table(df,index=["Manager","Rep"],values=["Price"],
               columns=["Product"],aggfunc=[np.sum],fill_value=0)

Add Quantity to the values list.

In [ ]:
pd.pivot_table(df,index=["Manager","Rep"],values=["Price","Quantity"],
               columns=["Product"],aggfunc=[np.sum],fill_value=0)

## Move items to the index 

This gives a different visual representation. 

Remove Product from the columns and add to the index .

In [ ]:
pd.pivot_table(df,index=["Manager","Rep","Product"],
               values=["Price","Quantity"],aggfunc=[np.sum],fill_value=0)

## Add some totals

Set the `margins` parameter to **True** .

The totals are in the last column `All`

In [ ]:
pd.pivot_table(df,index=["Manager","Rep","Product"],
               values=["Price","Quantity"],
               aggfunc=[np.sum,np.mean],fill_value=0, margins=True)

## Different Functions for Different Values

If you wenat to have different aggregate functions to different values, use a dictionary

For example
> **len** for the `Quuantity` value <BR>
> **np.sum** for the `Price` column <BR>

In [ ]:
funcs = {
    "Quantity":len,
    "Price":np.sum
}

pd.pivot_table(df,index=["Manager","Status"],columns=["Product"],values=["Quantity","Price"],
               aggfunc=funcs,fill_value=0)


It is also possible to provide a list of aggfunctions to apply to each value too:

For example <BR>
> **len** for `Quantity` values  <BR>
> **np.sum** and **np.mean** for the `Price` value  <BR>



In [ ]:
funcs = {
    "Quantity":len,
    "Price":[np.sum,np.mean]
}

table = pd.pivot_table(df,index=["Manager","Status"],columns=["Product"],values=["Quantity","Price"],
               aggfunc={"Quantity":len,"Price":[np.sum,np.mean]},fill_value=0)
table

## Advanced Pivot Table Filtering

Once you have generated your data, it is in a DataFrame so you can filter on it using your standard DataFrame functions.

This uses ther `query` method on a DataFrame

For example - filter for the manager equal to Debra Hanley

In [ ]:
is_mgr_Deb_Han = 'Manager == ["Debra Henley"]'

table.query(is_mgr_Deb_Han)

All of our pending and won deals.

In [ ]:
pending_or_won = 'Status == ["pending","won"]'

table.query(pending_or_won)

## Add some colour

### High Low colours

In [ ]:
funcs = {
    "Quantity":len,
    "Price":[np.sum,np.mean]
}

table = pd.pivot_table(df,index=["Manager","Status"],columns=["Product"],values=["Quantity","Price"],
               aggfunc={"Quantity":len,"Price":[np.sum,np.mean]},fill_value=0)

table.style.highlight_max(color='lightgreen').highlight_min(color='red')

### Gradient Colours

In [ ]:
table.style.background_gradient(cmap='Blues')

# Cheat - Sheet

<img style="float: center;" width="720" src="../Images/pivot-table.png">

# Crosstabs

`pandas.crosstab()` is another function that builds summary data

The pandas crosstab function builds a cross-tabulation table that can show the frequency with which certain groups of data appear. 



## Import the data

For this demo, only interested in the following subset of car manufacturers
>
> "toyota","nissan","mazda", "honda", "mitsubishi", "subaru", "volkswagen", "volvo"
>

In [ ]:
import pandas as pd
import seaborn as sns

# Define the headers since the data does not have any
headers = ["symboling", "normalized_losses", "make", "fuel_type", "aspiration",
           "num_doors", "body_style", "drive_wheels", "engine_location",
           "wheel_base", "length", "width", "height", "curb_weight",
           "engine_type", "num_cylinders", "engine_size", "fuel_system",
           "bore", "stroke", "compression_ratio", "horsepower", "peak_rpm",
           "city_mpg", "highway_mpg", "price"]

# Read in the CSV file and convert "?" to NaN
df_raw = pd.read_csv(filepath_or_buffer='../Data/cars.csv',
                     header=None, names=headers, na_values="?" )

# Define a list of models that we want to review
models = ["toyota","nissan","mazda", "honda", "mitsubishi", "subaru", "volkswagen", "volvo"]

# Create a copy of the data with only the top 8 manufacturers
df = df_raw[df_raw.make.isin(models)].copy()

## How many different body styles these car makers made

The crosstab function can operate on numpy arrays, series or columns in a dataframe. 

Here, `df.make` is set to be the crosstab index and `df.body_style` is the crosstab’s columns. 

Pandas does that work behind the scenes to count how many occurrences there are of each combination. 

For example, in this data set Volvo makes 8 sedans and 3 wagons.

In [ ]:
pd.crosstab(df['make'], df['body_style'])

### Use a groupby followed by an unstack to get the same results

In [ ]:
df.groupby(['make', 'body_style'])['body_style'].count().unstack().fillna(0)

### Use a pivot_table followed by an unstack to get the same results

In [ ]:
df.pivot_table(index=df['make'], columns=df['body_style'], aggfunc={'body_style':len}, fill_value=0)

## Add Subtotals

use the `margins` keyword:

In [ ]:
pd.crosstab(index=df['make'], columns=df['num_doors'], margins=True, margins_name="Total")

## Add aggregation

use the `aggfunc` parameter

specify the columns using the `values` parameter

In [ ]:
pd.crosstab(index=df['make'], columns=df['body_style'], values=df.curb_weight, aggfunc='mean').round(0)

## Normalize

The precentage time each combination occurs

In [ ]:
pd.crosstab(index=df['make'], columns=df['body_style'], normalize=True)

>
>The table above shows that 2.3% of the total population are Toyota hardtops and 6.25% are Volvo sedans.
>

### Normalize on colums only

In [ ]:
pd.crosstab(index=df['make'], columns=df['body_style'], normalize='columns')

>
> This table shows that 50% of the convertibles are made by Toyota and the other 50% by Volkswagen.
>

### Normalize on rows only

In [ ]:
pd.crosstab(index=df['make'], columns=df['body_style'], normalize='index')

>
> The above table shows that of the Mitsubishi cars in this dataset, 69.23% are hatchbacks and the remainder (30.77%) are sedans.
>

# Grouping

An extremely useful feature is to pass in multiple dataframe columns and pandas does all the grouping for you. 

For instance, to see how the data is distributed by front wheel drive (fwd) and rear wheel drive (rwd), 
 - include the `drive_wheels` column by including it in the list of valid columns in the second argument to the crosstab .

In [ ]:
cols = [ df['body_style'], df['drive_wheels'] ]

pd.crosstab(index=df['make'], columns=cols)

# Group the index

Included the specific rownames and colnames to include in the output. This is purely for display purposes but can be useful if the column names in the dataframe are not very specific.

Use `dropna=False` at the end of the function call. This is =to make sure to include all the rows and columns even if they had all 0’s. 

If it was not include it, then the final Volvo, two door row would have been omitted from the table.


In [ ]:
cols = [ df['body_style'], df['drive_wheels'] ]
idx  = [ df['make'], df['num_doors'] ]

pd.crosstab(index=idx, columns=cols,
            rownames=['Auto Manufacturer', "Doors"],
            colnames=['Body Style', "Drive Type"],
            dropna=False)

# Visualizing

Create a heatmap using `seaborn.heatmap()` function.

In [ ]:
cols = [ df['body_style'], df['drive_wheels'] ]
idx  = [ df['make'], df['num_doors'] ]
crosstab = pd.crosstab([df.make, df.num_doors], [df.body_style, df.drive_wheels])

sns.heatmap(data=crosstab, cmap="YlGnBu", annot=True, cbar=True)

# Cheat - Sheet

<img style="float: center;" width="1440" src="../Images/crosstab.png">